In [1]:
import tensorflow as tf
import numpy as np
import csv
import random

In [2]:
collectionFilePath = 'F:\MojiPop\collection.csv'
usersFilePath = 'F:\MojiPop\\users.csv'
caricaturesFilePath = 'F:\MojiPop\caricatures.csv'

In [3]:
def readCsv(filePath):
    csvData = []
    with open(filePath) as file:
        csv_reader = csv.reader(file)
        csvHeader = next(csv_reader)
        for row in csv_reader:
            csvData.append(row)
    return csvData

In [4]:
collectionRawData = readCsv(collectionFilePath)
#print(len(collectionRawData))
collectionData = []
for row in collectionRawData:
    tempData = row[0].split(';')
    collectionData.append(tempData)
print(collectionData[0])
    

['65702', '121000900710 ']


In [5]:
usersRawData = readCsv(usersFilePath)
#print(len(usersRawData))
usersData = []
for row in usersRawData:
    tempData = row[0].split(';')
    tempData[2] = int(tempData[2])
    usersData.append(tempData)
#print(usersData[2])

usersMatrix = []
for row in usersData:
    gender_F = 1 if(row[1]=='F') else 0
    gender_M = 1 if(row[1]=='M') else 0
    gender_UNK = 1 if(row[1]=='') else 0
    age_UNK = 1 if(row[2]<=0) else 0
    age_BABY = 1 if(row[2]>0 and row[2]<=3) else 0
    age_TEEN = 1 if(row[2]>3 and row[2]<=14) else 0
    age_YOUNG = 1 if(row[2]>14 and row[2]<=29) else 0
    age_MIDDLE = 1 if(row[2]>29 and row[2]<=55) else 0
    age_OLD = 1 if(row[2]>55) else 0
    lang_EN = 1 if(row[3]=='en') else 0
    lang_PT = 1 if(row[3]=='pt') else 0
    lang_ZH_TW = 1 if(row[3]=='zh_tw') else 0
    lang_FR = 1 if(row[3]=='fr') else 0
    lang_KO = 1 if(row[3]=='ko') else 0
    lang_RU = 1 if(row[3]=='ru') else 0
    lang_ES = 1 if(row[3]=='es') else 0
    lang_AR = 1 if(row[3]=='ar') else 0
    lang_ZH = 1 if(row[3]=='zh') else 0
    lang_JA = 1 if(row[3]=='ja') else 0
    tempData = [gender_F, gender_M, gender_UNK, 
                age_UNK, age_BABY, age_TEEN, age_YOUNG, age_MIDDLE, age_OLD,
                lang_EN, lang_PT, lang_ZH_TW, lang_FR, lang_KO, lang_RU, lang_ES, lang_AR, lang_ZH, lang_JA]
    usersMatrix.append(tempData)
usersMatrix = np.array(usersMatrix, dtype=float)
#print(usersMatrix)
print(usersMatrix.shape)

usersIndex = []
for i in range(len(usersData)):
    usersIndex.append(usersData[i][0])
#print(usersIndex[0])


(4642, 19)


In [6]:
caricaturesRawData = readCsv(caricaturesFilePath)
#print(caricaturesRawData[0])

caricaturesOneHot = tf.one_hot(np.linspace(0,len(caricaturesRawData)-1,len(caricaturesRawData)),len(caricaturesRawData))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    caricaturesMatrix = sess.run(caricaturesOneHot)
print(np.size(caricaturesMatrix,0))

caricaturesIndex = []
for i in range(len(caricaturesRawData)):
    caricaturesIndex.append(caricaturesRawData[i][0])
#print(caricaturesIndex[0])


1280


In [7]:
collectionMatrix = np.zeros([4642,1280])

for collection in collectionData:
    userNo = usersIndex.index(collection[0])
    caricatureNo = caricaturesIndex.index(collection[1])
    #print(caricaturesMatrix[caricatureNo].sum())
    collectionMatrix[userNo] = collectionMatrix[userNo] + caricaturesMatrix[caricatureNo]
print(collectionMatrix.shape[0])
print(collectionMatrix.sum(axis=1))

4642
[ 1. 16.  1. ... 11. 10.  7.]


In [8]:
x0 = tf.placeholder(tf.float32, [None, 19])
x1 = tf.placeholder(tf.float32, [None, 1280])
y = tf.placeholder(tf.float32, [None, 1280])
dropout_rate = tf.placeholder(tf.float32)
hyperP = 200
generalization_rate = 1280*0.20


v0 = tf.get_variable('v0', [x0.get_shape()[1], hyperP], 
                    initializer = tf.random_normal_initializer(0.00,0.01))
v1 = tf.get_variable('v1', [x1.get_shape()[1], hyperP],
                    initializer = tf.random_normal_initializer(0.00,0.01))
embedding0 = tf.matmul(x0, v0)
embedding1 = tf.matmul(x1, v1)
embedding = tf.concat([embedding0, embedding1], 1)

# FM part
w0 = tf.get_variable('w0', [tf.concat([x0, x1],1).get_shape()[1], hyperP], 
                    initializer = tf.random_normal_initializer(0.00, 1.00))
b0 = tf.get_variable('b0', [hyperP],
                    initializer = tf.random_normal_initializer(0.00, 1.00))
linear_term = tf.add(tf.matmul(tf.concat([x0,x1],1), w0), b0)

interaction_term = tf.multiply(embedding0, embedding1)
y_fm = tf.concat([linear_term, interaction_term], axis=1)


#DNN part
embedding = tf.nn.dropout(embedding, dropout_rate)
glorot = tf.sqrt(2.0/(tf.cast(embedding.get_shape()[1], tf.float32) + hyperP))
w1 = tf.get_variable('w1', [embedding.get_shape()[1], hyperP],
                    initializer = tf.random_normal_initializer(0.00, glorot))
b1 = tf.get_variable('b1', [hyperP], 
                    initializer = tf.random_normal_initializer(0.00, glorot))
y_hidden1 = tf.nn.relu(tf.matmul(embedding, w1) + b1)
y_hidden1 = tf.nn.dropout(y_hidden1, dropout_rate)
glorot = tf.sqrt(1/hyperP)
w2 = tf.get_variable('w2', [hyperP, hyperP],
                    initializer = tf.random_normal_initializer(0.00, glorot))
b2 = tf.get_variable('b2', [hyperP], 
                    initializer = tf.random_normal_initializer(0.00, glorot))
y_hidden2 = tf.nn.relu(tf.matmul(y_hidden1, w2) + b2)
y_hidden2 = tf.nn.dropout(y_hidden2, dropout_rate)
w3 = tf.get_variable('w3', [hyperP, hyperP],
                    initializer = tf.random_normal_initializer(0.00, glorot))
b3 = tf.get_variable('b3', [hyperP], 
                    initializer = tf.random_normal_initializer(0.00, glorot))
y_dnn = tf.nn.relu(tf.matmul(y_hidden2, w3) + b3)
y_dnn = tf.nn.dropout(y_dnn, dropout_rate)

#out
glorot = tf.sqrt(2.0/(tf.cast(y_fm.get_shape()[1], tf.float32) + 
                      tf.cast(y_dnn.get_shape()[1], tf.float32) + 1))
w_sparse = tf.get_variable('w_sparse', [y_fm.get_shape()[1]+y_dnn.get_shape()[1], y.get_shape()[1]],
                    initializer = tf.random_normal_initializer(0.00, glorot))
b_sparse = tf.get_variable('b_sparse', [y.get_shape()[1]], 
                    initializer = tf.constant_initializer(0.01))
y_all = tf.concat([y_fm, y_dnn], 1)
y_out = tf.add(tf.matmul(y_all, w_sparse), b_sparse)
y_out_prob = tf.nn.sigmoid(y_out)

#loss
y_out_generalization = tf.multiply(y_out_prob, y)
loss = generalization_rate * tf.reduce_mean(tf.square(y_out_generalization - y)) +  tf.reduce_mean(tf.square(y_out_prob - y))

#accuracy
predict = tf.cast(tf.greater(y_out_prob, 0.5), tf.float32)
#correct_prediction = tf.equal(predict, y)
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
predict_generalization = tf.multiply(predict, y)
accuracy = tf.count_nonzero(predict_generalization)/tf.count_nonzero(y)

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-2).minimize(loss)

In [9]:
echos = 10
batch_size = 500
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(echos):
        print('echos: %d' % i);
        start_index = 0
        while (start_index+batch_size < collectionMatrix.shape[0]*0.9):
            batch_x0 = usersMatrix[start_index: start_index+batch_size]
            batch_x1 = collectionMatrix[start_index: start_index+batch_size]
            batch_y = collectionMatrix[start_index: start_index+batch_size]
            index = [i for i in range(batch_size)]
            random.shuffle(index)
            batch_x0 = batch_x0[index]
            batch_x1 = batch_x1[index]
            batch_y = batch_y[index]
            loss_val, acc_val, _ = sess.run(
                [loss, accuracy, train_op], feed_dict = {x0:batch_x0, x1:batch_x1, y:batch_y, dropout_rate:0.5})
            print ('[Train] loss=%4.5f  accuracy:%4.5f'% (loss_val,acc_val))
            start_index += batch_size
        test_x0 = usersMatrix[start_index:]
        test_x1 = collectionMatrix[start_index:]
        test_y = collectionMatrix[start_index:] 
        test_loss, test_acc = sess.run(
            [loss, accuracy], feed_dict = {x0:test_x0, x1:test_x1, y:test_y, dropout_rate:1.0})
        print('[Test] loss=%4.5f  accuracy:%4.5f'% (test_loss, test_acc))
    y_values, predict_resource = sess.run(
        [y_out_prob, predict], feed_dict = {x0: usersMatrix, x1:collectionMatrix, y:collectionMatrix, dropout_rate:1.0})
    print(predict_resource.sum(axis=1))

echos: 0
[Train] loss=1.70822  accuracy:0.50155
[Train] loss=1.27882  accuracy:0.63563
[Train] loss=1.15930  accuracy:0.77215
[Train] loss=0.83232  accuracy:0.84737
[Train] loss=0.65179  accuracy:0.88623
[Train] loss=0.58694  accuracy:0.89964
[Train] loss=0.50388  accuracy:0.90006
[Train] loss=0.49064  accuracy:0.91269
[Test] loss=0.46923  accuracy:0.91907
echos: 1
[Train] loss=0.46189  accuracy:0.95196
[Train] loss=0.43956  accuracy:0.95331
[Train] loss=0.41225  accuracy:0.96563
[Train] loss=0.37157  accuracy:0.96551
[Train] loss=0.33037  accuracy:0.96457
[Train] loss=0.32875  accuracy:0.95287
[Train] loss=0.29561  accuracy:0.93741
[Train] loss=0.28717  accuracy:0.94531
[Test] loss=0.34867  accuracy:0.91135
echos: 2
[Train] loss=0.27332  accuracy:0.97371
[Train] loss=0.27644  accuracy:0.96807
[Train] loss=0.26147  accuracy:0.97788
[Train] loss=0.24724  accuracy:0.97239
[Train] loss=0.23126  accuracy:0.96913
[Train] loss=0.22786  accuracy:0.96336
[Train] loss=0.20423  accuracy:0.95290


In [15]:
recommend_resource = np.subtract(predict_resource, collectionMatrix)
recommend_resource[recommend_resource < 0] = 0
recommend_resource_indices = np.nonzero(recommend_resource)
print(recommend_resource_indices)

(array([   1,    1,    1, ..., 4641, 4641, 4641], dtype=int64), array([   9,   13,   34, ...,  657,  790, 1222], dtype=int64))


In [16]:
recommend_resource_indices[0].shape

(345481,)